In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('browsers_visits.csv')
data.head()

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [3]:
shares = data.groupby(['date', 'device', 'browser']).agg({'visits': {'share': 'sum'}})
shares.head(12)

visits
                                       share
date       device  browser                  
2016-06-06 desktop androidbrowser       5086
                   chrome          228077636
                   chromemobile         7032
                   edge              8693600
                   firefox          75770521
                   msie             51416938
                   opera            65942946
                   operamobile           830
                   other            13527964
                   safari            7117855
                   ucweb              290045
                   yandexbrowser   100392221

In [4]:
shares = shares.groupby(level=[0,1]).apply(lambda x: 100 * x / float(x.sum()))
shares.head(12)

visits
                                       share
date       device  browser                  
2016-06-06 desktop androidbrowser   0.000923
                   chrome          41.375178
                   chromemobile     0.001276
                   edge             1.577091
                   firefox         13.745402
                   msie             9.327460
                   opera           11.962598
                   operamobile      0.000151
                   other            2.454085
                   safari           1.291238
                   ucweb            0.052617
                   yandexbrowser   18.211983

In [5]:
shares_all = data.groupby(['date', 'browser']).agg({'visits': {'share': 'sum'}})
shares_all = shares_all.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
shares_all.head(12)

visits
                               share
date       browser                  
2016-06-06 androidbrowser   3.980815
           chrome          28.956725
           chromemobile    10.033126
           edge             1.187771
           firefox          9.623200
           mobilesafari     7.733731
           msie             6.601638
           opera            8.405954
           operamini        0.868755
           operamobile      0.706214
           other            3.500018
           safari           0.904302

In [6]:
device_types = np.append(data.device.unique(), 'all')
browsers = data.browser.unique()

N = len(device_types)
M = len(browsers)

data_plot = []
drop_menus = []

for i in range(N):
    
    for j in range(M):
        
        device_type = device_types[i]
        browser = browsers[j]
        
        if i == N-1:
            df_part = shares_all
        else:
            df_part = shares[shares.index.get_level_values('device') == device_type]
            
        df_part = df_part[df_part.index.get_level_values('browser') == browser]

        trace = go.Scatter(
            x = df_part.index.get_level_values('date'),
            y = df_part.visits.share,
            mode = 'lines',
            name = browser,
            visible = (i == 0)
        )

        data_plot.append(trace)
    
for i in range(N):
    drop_menus.append(
        dict(
            args=['visible', [False]*M*i + [True]*M + [False]*M*(N-1-i)],
            label= device_types[i],
            method='restyle'
        )
    )

layout = go.Layout(
    title='Browsers shares by device type',
    updatemenus=list([
        dict(
            x = -0.1,
            y = 1,
            yanchor = 'top',
            buttons = drop_menus
        )
    ]),
)

fig = go.Figure(data = data_plot, layout = layout)
iplot(fig, show_link = False)